In [1]:
import pandas as pd

train = pd.read_csv('./datas/train_ver4', index_col=[0])
train.tail()

,comments,hate_label,news_title,comment_pos,title_pos
7891,"힘내세요,응원합니다",none,"허지웅 허투루 넘길 말 없었다,솔직하게 드러냈던 속사정","[('힘내세요', 'Verb'), (',', 'Punctuation'), ('응원'...","[('허지웅', 'Noun'), ('허투루', 'Noun'), ('넘길', 'Ver..."
7892,"힘내세요,삼가 고인의 명복을 빕니다",none,이혜경 오 캐롤 공연 중 남편 오정욱 부고 오열 속 발인 종합,"[('힘내세요', 'Verb'), (',', 'Punctuation'), ('삼가'...","[('이혜경', 'Noun'), ('오', 'Noun'), ('캐롤', 'Noun'..."
7893,힘내세용 항상 응원합니닷,none,설경구 송윤아 아들과 즐거운 하루 전 엄마니까요,"[('힘내세용', 'Verb'), ('항상', 'Noun'), ('응원', 'Nou...","[('설경구', 'Noun'), ('송윤아', 'Noun'), ('아들', 'Nou..."
7894,"힘내소연기로 답해요,나도 53살 인데 이런일 저런일 다 있더라구요,인격을 믿습니다,홨팅",none,SC현장연예인 인생 협박 유감 미소잃은 최민수 보복운전 혐의 2차 공판종합,"[('힘내소', 'Verb'), ('연기', 'Noun'), ('로', 'Josa'...","[('SC', 'Alpha'), ('현장', 'Noun'), ('연예인', 'Nou..."
7895,힘들면 관뒀어야지 그게 현명한거다,none,단독스태프 사망사고 서른이지만 결국 오늘 촬영 취소,"[('힘들면', 'Adjective'), ('관', 'Noun'), ('뒀어야지',...","[('단독', 'Noun'), ('스태프', 'Noun'), ('사망', 'Noun..."


In [2]:
none = train[train['hate_label'] == 'none']
hate = train[train['hate_label'] == 'hate']
offensive = train[train['hate_label'] == 'offensive']

In [3]:
import pandas as pd
from konlpy.tag import Okt; t = Okt()
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
def t_tokenizer(text):
    tokens_ko = t.morphs(text)
    return tokens_ko

## hate_label == none data

In [5]:
tfidf_vect = TfidfVectorizer(tokenizer=t_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(none['comments'])
tfidf_none = tfidf_vect.transform(none['comments'])
none_avg = np.mean(tfidf_none)

## hate_label == offensive data

In [6]:
tfidf_vect = TfidfVectorizer(tokenizer=t_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(offensive['comments'])
tfidf_offensive = tfidf_vect.transform(offensive['comments'])
offensive_avg = np.mean(tfidf_offensive)

## hate_label == hate data

In [7]:
tfidf_vect = TfidfVectorizer(tokenizer=t_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(hate['comments'])
tfidf_hate = tfidf_vect.transform(hate['comments'])
hate_avg = np.mean(tfidf_hate)

In [8]:
dev = pd.read_csv('./datas/dev.hate.csv')
dev.rename(columns={'label': 'hate_label'}, inplace=True)
dev.tail()

,comments,hate_label
466,지현우 범죄 저지르지 않았나요?,offensive
467,여자인생 망칠 일 있나 ㅋㅋ,hate
468,근데 전라도에서 사고가 났는데 굳이 서울까지 와서 병원에 가느 이유는?,offensive
469,"할매젖x, 뱃살x, 몸매 s라인, 유륜은 적당해야됨(너무크거나 너무 작아도 x), ...",hate
470,남자가 잘못한거라면... 반성도 없다면...나였다면 ... 여자처럼 아주 못되게 할...,none


In [31]:
# updated train_ver4
none_avg, hate_avg, offensive_avg

(0.0008316164304716909, 0.0013060816402752783, 0.0010750158738200074)

In [10]:
# train_ver1
# none_avg, hate_avg, offensive_avg
# (0.0008018106712232337, 0.0012499658321840192, 0.0010364766155626834)

In [11]:
tfidf_dev = tfidf_vect.transform(dev['comments'])
print(tfidf_dev[0])

  (0, 1753)	0.23593658891749042
  (0, 1314)	0.6014303962951241
  (0, 1120)	0.5544830070759627
  (0, 1016)	0.524560767963319


In [12]:
mean = []
for each in tfidf_dev:
    mean.append(np.mean(each))
dev['mean_dis'] = mean
dev.tail()

,comments,hate_label,mean_dis
466,지현우 범죄 저지르지 않았나요?,offensive,0.000389
467,여자인생 망칠 일 있나 ㅋㅋ,hate,0.000845
468,근데 전라도에서 사고가 났는데 굳이 서울까지 와서 병원에 가느 이유는?,offensive,0.001340
469,"할매젖x, 뱃살x, 몸매 s라인, 유륜은 적당해야됨(너무크거나 너무 작아도 x), ...",hate,0.001156
470,남자가 잘못한거라면... 반성도 없다면...나였다면 ... 여자처럼 아주 못되게 할...,none,0.001630


In [21]:
import pandas as pd

train = pd.read_csv('./datas/train_ver4', index_col=[0])
train.tail()

,comments,hate_label,news_title,comment_pos,title_pos
7891,"힘내세요,응원합니다",none,"허지웅 허투루 넘길 말 없었다,솔직하게 드러냈던 속사정","[('힘내세요', 'Verb'), (',', 'Punctuation'), ('응원'...","[('허지웅', 'Noun'), ('허투루', 'Noun'), ('넘길', 'Ver..."
7892,"힘내세요,삼가 고인의 명복을 빕니다",none,이혜경 오 캐롤 공연 중 남편 오정욱 부고 오열 속 발인 종합,"[('힘내세요', 'Verb'), (',', 'Punctuation'), ('삼가'...","[('이혜경', 'Noun'), ('오', 'Noun'), ('캐롤', 'Noun'..."
7893,힘내세용 항상 응원합니닷,none,설경구 송윤아 아들과 즐거운 하루 전 엄마니까요,"[('힘내세용', 'Verb'), ('항상', 'Noun'), ('응원', 'Nou...","[('설경구', 'Noun'), ('송윤아', 'Noun'), ('아들', 'Nou..."
7894,"힘내소연기로 답해요,나도 53살 인데 이런일 저런일 다 있더라구요,인격을 믿습니다,홨팅",none,SC현장연예인 인생 협박 유감 미소잃은 최민수 보복운전 혐의 2차 공판종합,"[('힘내소', 'Verb'), ('연기', 'Noun'), ('로', 'Josa'...","[('SC', 'Alpha'), ('현장', 'Noun'), ('연예인', 'Nou..."
7895,힘들면 관뒀어야지 그게 현명한거다,none,단독스태프 사망사고 서른이지만 결국 오늘 촬영 취소,"[('힘들면', 'Adjective'), ('관', 'Noun'), ('뒀어야지',...","[('단독', 'Noun'), ('스태프', 'Noun'), ('사망', 'Noun..."


In [22]:
from konlpy.tag import Okt; t = Okt()
from sklearn.feature_extraction.text import TfidfVectorizer

def t_tokenizer(text):
    tokens_ko = t.morphs(text)
    return tokens_ko
    
tfidf_vect = TfidfVectorizer(ngram_range=(1,2), min_df=3, max_df=0.9)

In [23]:
comments_tokens = train['comments'].apply(t_tokenizer)

In [24]:
comments_for_vectorize = []
for comment in comments_tokens:
    sentence = ''
    for word in comment:
        sentence = sentence + ' ' + word
    comments_for_vectorize.append(sentence)

In [25]:
X = tfidf_vect.fit_transform(comments_for_vectorize)
X.shape

(7896, 5379)

In [26]:
dev = pd.read_csv('./datas/dev.hate.csv')
dev.rename(columns={'label': 'hate_label'}, inplace=True)
dev.tail()

,comments,hate_label
466,지현우 범죄 저지르지 않았나요?,offensive
467,여자인생 망칠 일 있나 ㅋㅋ,hate
468,근데 전라도에서 사고가 났는데 굳이 서울까지 와서 병원에 가느 이유는?,offensive
469,"할매젖x, 뱃살x, 몸매 s라인, 유륜은 적당해야됨(너무크거나 너무 작아도 x), ...",hate
470,남자가 잘못한거라면... 반성도 없다면...나였다면 ... 여자처럼 아주 못되게 할...,none


In [27]:
dev_tokens = dev['comments'].apply(t_tokenizer)

In [28]:
dev_for_vectorize = []
for comment in dev_tokens:
    sentence = ''
    for word in comment:
        sentence = sentence + ' ' + word
    dev_for_vectorize.append(sentence)

In [29]:
dev = tfidf_vect.transform(dev_for_vectorize)
dev.shape

(471, 5379)

In [30]:
import scipy as sp

def dist_raw(v1, v2):
    delta = v1 - v2
    return sp.linalg.norm(delta.toarray())

In [31]:
import scipy as sp

def dist_norm(v1, v2):
    v1_norm = v1 / sp.linalg.norm(v1.toarray())
    v2_norm = v2 / sp.linalg.norm(v2.toarray())
    delta = v1_norm - v2_norm
    return sp.linalg.norm(delta.toarray())

In [37]:
length = list(range(train.shape[0]))
i = 0

dis = []
while i < len(length):
    for d in dev:
        dis.append(dist_norm(X[i], d))
    i += 1

In [44]:
min_dis = []
for each in dis:
    min_dis.append(np.min(each))

In [45]:
len(min_dis)

3719016

In [36]:
mean = []
for each in tfidf_dev:
    mean.append(np.mean(each))
dev['mean_dis'] = mean

7896

In [115]:
dis = []
for x in X:
    each_dis = [dist_raw(x, d) for d in dev]
    dis.append(each_dis)
    print(dis)

2135623730951, 1.4142135623730951, 1.4142135623730951, 1.4142135623730951, 1.4142135623730951, 1.414213562373095, 1.4142135623730951, 1.414213562373095, 1.4142135623730951, 1.4142135623730951, 1.4142135623730951, 1.4142135623730951, 1.4142135623730951, 1.0, 1.4142135623730951, 1.4142135623730951, 1.4142135623730951, 1.4142135623730951, 1.4142135623730951, 1.4142135623730951, 1.414213562373095, 1.4142135623730951, 1.4142135623730951, 1.4142135623730951, 1.4142135623730951, 1.4142135623730951, 1.4142135623730951, 1.4142135623730951, 1.4142135623730951, 1.4142135623730951, 1.4142135623730951, 1.414213562373095, 1.4142135623730951, 1.354222852839224, 1.4142135623730951, 1.4142135623730951, 1.4142135623730951, 1.4142135623730951, 1.4142135623730951, 1.4142135623730951, 1.4142135623730951, 1.4142135623730951, 1.4142135623730951, 1.4142135623730951, 1.4142135623730951, 1.4142135623730951, 1.4142135623730951, 1.4142135623730951, 1.4142135623730951, 1.4142135623730951, 1.4142135623730951, 1.414

KeyboardInterrupt: 